# Random Forest
In dit notebook zal het model Random Forest centraal staan. Hierbij worden er hard drives door het Random Forest model voorspeld.

## Libraries Importeren

In [5]:
import pandas as pd
import numpy as np
import pickle

from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, plot_confusion_matrix
from sklearn.tree import export_graphviz

## Random Forest model

Het random forest model is een model gebruikt voor o.a. classificatie. Het is een verzameling van verschillende decision trees. Door gebruik te maken van bagging en feature randomness op deze decision trees wordt er een “forest” gecreeërd die bestaat uit meerdere trees die niet gecorreleerd zijn met elkaar. Deze forest is in de meeste gevallen meer accuraat dan één enkele tree

### Random Forest Classification

De eerste stap zal zijn het kunnen predicten of een drive een failure heeft of niet aan de hand van de SMART-waardes. Deze SMART-waardes zijn 5, 9, 187, 188, 194, 197 en 198. Hiervoor is de Random Forest classification het beste doordat we te maken hebben met een boolean prediction value (het is een failure of geen failure).

### Data importeren
De data die gebruikt wordt is de data die gegenereerd is in het notebook `2 - sample analysing` (de dataset zonder 100% NaN waardes).

In [4]:
sample = pd.read_csv('../data/sample_without_100nan.csv')

### Data Cleaning
In de huidige dataset zitten er nog een aantal kolommen die geen toegevoegde waarde hebben voor het predicten van de failure. Deze kolommen zijn datum, model en serienummer. Hierdoor zullen deze eruit gehaald worden. Om toch de originele dataset later te kunnen gebruiken is er een keuze gemaakt om eerst een kopie te maken en daarmee verder te gaan.

In [6]:
copy_sample = sample.copy()

In [7]:
del copy_sample['date']
del copy_sample['serial_number']
del copy_sample['model']

Een ander gedeelte van het cleanen van data is het opvullen van lege waardes of het verwijderen van rows waar lege waardes in zitten. In 2 - sample analysing is te zien dat her percentage van lege waardes voor alle modellen minder dan 1% is. Hierdoor is er een keuze gemaakt om de rijen te verwijderen.

In [8]:
copy_sample.shape

(22008389, 8)

In [9]:
assert isinstance(copy_sample, pd.DataFrame), "df needs to be a pd.DataFrame"
copy_sample.dropna(inplace=True)
indices_to_keep = ~copy_sample.isin([np.nan, np.inf, -np.inf]).any(1)
copy_sample[indices_to_keep].astype(np.float64)

,failure,smart_5_raw,smart_9_raw,smart_187_raw,smart_188_raw,smart_194_raw,smart_197_raw,smart_198_raw
0,0.0,0.0,35462.0,0.0,0.0,22.0,0.0,0.0
1,0.0,0.0,12494.0,0.0,0.0,28.0,0.0,0.0
2,0.0,0.0,9544.0,0.0,0.0,29.0,0.0,0.0
3,0.0,0.0,13098.0,0.0,0.0,25.0,0.0,0.0
4,0.0,0.0,23427.0,0.0,0.0,32.0,0.0,0.0
...,...,...,...,...,...,...,...,...
22008384,0.0,0.0,1142.0,0.0,0.0,33.0,0.0,0.0
22008385,0.0,8.0,39697.0,0.0,0.0,24.0,0.0,0.0
22008386,0.0,0.0,1223.0,0.0,0.0,20.0,0.0,0.0
22008387,0.0,0.0,16158.0,0.0,0.0,21.0,0.0,0.0


Zoals hierboven en hieronder te zien was de shape eerst 22008389 en na het deleten van de rows 22002242. Dat is een verschil van ongeveer 6000 rows. Dat is niks op een dataset van 22 miljoen rows.

Daarnaast is al eerder aangegeven dat failure de kolom zal zijn die predict gaat worden. Hieronder wordt de failure op de Y axis gezet.

In [11]:
class_X = copy_sample.drop(['failure'], axis=1).values
class_Y = copy_sample['failure'].values

class_X.shape, class_Y.shape

((22002242, 7), (22002242,))

### Opsplitsen trainings- en validatieset
Voor het opsplitsen van de data in een trainingsset en in een validatieset gebruiken we `Scikit-learn`. Een model wordt gebruikt om te trainen, en de andere wordt gebruikt om te testen.

De data wordt opgesplit in een 80/20 split.

In [12]:
class_X_train, class_X_test, class_Y_train, class_Y_test = train_test_split(class_X, class_Y, test_size=0.2)

Daarna is nog een keer de shape te zien van zowel de Y als de X axis.

In [ ]:
class_X_train.shape, class_Y_train.shape

In [ ]:
class_X_test.shape, class_Y_test.shape

### Trainen van het model
Hier wordt het model getrained. Dit wordt gedaan door middel van het Random Forest Classifier algoritme

In [ ]:
# rf_model = ensemble.RandomForestClassifier()
# rf_model.fit(class_X_train, class_Y_train)

Doordat het lang duurt om een model te trainen wordt het model opgeslagen zodat het meerdere keren gebruikt wordt. Dit wordt gedaan met pickle.

In [ ]:
# pickle.dump(rf_model, open("../models/random_forest2.pkl", "wb"))

Met deze code kan het model weer geladen worden.

In [16]:
rf_model_saved = pickle.load(open("../models/random_forest.pkl", "rb"))

In [17]:
rf_Y_pred = rf_model_saved.predict(class_X_test)

In [18]:
rf_acc = accuracy_score(class_Y_test, rf_Y_pred)

print('Accuracy:', rf_acc)

Accuracy: 0.9999850015305256


In [19]:
plot_confusion_matrix(rf_model_saved, class_X_test, rf_Y_pred)

C:\Users\r_grb\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function plot_confusion_matrix is deprecated; Function `plot_confusion_matrix` is deprecated in 1.0 and will be removed in 1.2. Use one of the class methods: ConfusionMatrixDisplay.from_predictions or ConfusionMatrixDisplay.from_estimator.
  warnings.warn(msg, category=FutureWarning)
